## $\text{Импортируем все необходимые библиотеки}$

In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.4 MB/s eta 0:00:00


In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 26.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import re
import math
import torch
import random
import shutil
import collections
import numpy as np
import pandas as pd
import torch.nn as nn
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from catboost import CatBoostClassifier
from transformers import AutoTokenizer
from sklearn.metrics import roc_auc_score, accuracy_score
from torchvision import transforms
from torchvision.models import resnext101_32x8d
from torch.nn.utils import clip_grad_norm_
from torch.optim import AdamW, lr_scheduler
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## $\text{Займемся формированием датасета для обучения нашей модели}$

In [ ]:
transform = transforms.Compose([
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

data = ImageFolder("/content/drive/MyDrive/data", transform=transform)

In [ ]:
train_size = int(0.8*len(data))
test_size= len(data) - train_size

train_data, test_data = random_split(data, [train_size, test_size])

In [ ]:
batch_size = 8

train_loader = DataLoader(train_data, 8, shuffle=True)
test_loader = DataLoader(test_data, 8)

In [ ]:
model = resnext101_32x8d(weights='ResNeXt101_32X8D_Weights.IMAGENET1K_V2')

Downloading: "https://download.pytorch.org/models/resnext101_32x8d-110c445d.pth" to /root/.cache/torch/hub/checkpoints/resnext101_32x8d-110c445d.pth
100%|██████████| 340M/340M [00:02<00:00, 173MB/s]


## $\text{Поскольку мы будем использовать предтренированную модель, то нам} \\
\text{нужно убрать последний слой, потому что он нужен для классификации на датасете ImageNet}$

In [ ]:
model = nn.Sequential(*list(model.children())[:-1]).to(device) # убираем последний слой

## $\text{Далее применим следующую идею:}\\
\text{Будем прогонять наши изображения через предтренированную модель, после чего полученные эмбэддинги будем подавать} \\
\text{в градиентный бустинг, и вот уже его будем обучать на классификацию}$

In [ ]:
train_embeddings = []
train_labels = []

test_embeddings = []
test_labels = []

model.eval()

with torch.no_grad():
    for images, labels in tqdm(train_loader):
        out = model(images.to(device))
        # print(out)
        # print(f"out.shape: {out.shape}")
        # print(f"out_reshape: {out.view(out.size(0), -1)}")

        train_embeddings.append(out.view(out.size(0), -1).cpu().numpy())
        train_labels.append(labels.cpu().numpy())

100%|██████████| 60/60 [01:59<00:00,  1.99s/it]


In [ ]:
with torch.no_grad():
    for images, labels in tqdm(test_loader):
        out = model(images.to(device))
        # print(out)
        # print(f"out.shape: {out.shape}")
        # print(f"out_reshape: {out.view(out.size(0), -1)}")

        test_embeddings.append(out.view(out.size(0), -1).cpu().numpy())
        test_labels.append(labels.cpu().numpy())

100%|██████████| 15/15 [00:31<00:00,  2.09s/it]


## $\text{Создадим выборки для того, чтобы их можно было подавать в градиентный бустинг}$

In [ ]:
X_train = np.concatenate(train_embeddings, axis=0)
y_train = np.concatenate(train_labels, axis=0)

X_test = np.concatenate(test_embeddings, axis=0)
y_test = np.concatenate(test_labels, axis=0)

In [ ]:
np.save('/content/drive/MyDrive/train_embeddings.npy', X_train.astype(np.float32))
np.save('/content/drive/MyDrive/train_labels.npy', y_train)

In [ ]:
np.save('/content/drive/MyDrive/test_embeddings.npy', X_test.astype(np.float32))
np.save('/content/drive/MyDrive/test_labels.npy', y_test)

## $\text{Далее обучим градиентный бустинг на классификацию наличия ж/д на фотографиях}$

In [ ]:
classifier = CatBoostClassifier(verbose=0)

In [ ]:
classifier.fit(X_train, y_train)

Learning rate set to 0.007531
0:	learn: 0.6888826	total: 643ms	remaining: 10m 42s
1:	learn: 0.6841975	total: 1.05s	remaining: 8m 42s
2:	learn: 0.6796417	total: 1.42s	remaining: 7m 53s
3:	learn: 0.6743387	total: 1.94s	remaining: 8m 3s
4:	learn: 0.6701516	total: 2.33s	remaining: 7m 42s
5:	learn: 0.6659640	total: 2.71s	remaining: 7m 28s
6:	learn: 0.6621385	total: 3.12s	remaining: 7m 22s
7:	learn: 0.6574818	total: 3.51s	remaining: 7m 15s
8:	learn: 0.6531692	total: 3.9s	remaining: 7m 9s
9:	learn: 0.6489267	total: 4.3s	remaining: 7m 5s
10:	learn: 0.6434832	total: 4.86s	remaining: 7m 17s
11:	learn: 0.6402176	total: 5.52s	remaining: 7m 34s
12:	learn: 0.6358547	total: 6.13s	remaining: 7m 45s
13:	learn: 0.6318379	total: 6.81s	remaining: 7m 59s
14:	learn: 0.6288997	total: 7.42s	remaining: 8m 7s
15:	learn: 0.6231522	total: 8.07s	remaining: 8m 16s
16:	learn: 0.6193882	total: 8.48s	remaining: 8m 10s
17:	learn: 0.6154182	total: 8.87s	remaining: 8m 3s
18:	learn: 0.6117853	total: 9.26s	remaining: 7m 57

## $\text{Далее проведем инференс нашей модели, посмотрим и оценим основные метрики классфикации и сделаем выводы}$

In [ ]:
outputs = classifier.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, outputs)
print(f"Accuracy = {accuracy}")

Accuracy = 0.8
